## IV. Tuning 

### Get data and desired features

#### Scoring function 
**How we select a model and how this reflects the model put into practice:**  
Ultimately, we care only that we correctly identify failed inspections among the inspections that the City of Chicago actually has the resources to carry out. So, we'd like our model to produce probabilities of passing or not passing -- the inspectors can then inspect establishments with highest probailities of not passing. We'd also like to punish or penalize the model for sending inspectors to inspect establishments that pass but that we predict not to pass with high certainty (or vice versa) (i.e. we'd like to be less wrong when we're not right). 

The log loss is a suitable objective function to optimize then. The log loss is the negative log likelihood of a Bernoulli random variable (in the 2-class setting, we'll justify this shortly): $$-\frac{1}{n}\sum_1^n y_i \log(p_i)-(1-y_i)\log(1-p_i)$$ for $n$ observations, where the $i$th observation is of correct class $y_i \in \{0,1\}$ which our model predicts with probability $p_i$. This achieves specifically this sort of penality (suppose $y_i=1$ and we predict this with probability of only $0.1$, this yields a value that approaches $-\infty$ rapidly).

When put into practice, this approach ranks inspections by probability of not passing, so inspectors can carry out inspections that appear at the top of this ranking. Selecting a model with minimal log loss (or maximum likelihood) essentially ensures this ranking is best, that inspectors have the highest chance of inspecting establishments that have commited a violation. Ultimately, in a given time frame with constrained resources, the City of Chicago cannot carry out all the inspections it has to do. There is a cutoff, so they will carry out the inspections at the top of this ranking. 

How can we get confidence in this approach (what does it look like in practice)? Suppose this model is run only one time (that's the best we can do, we don't have anymore data) and we use the resulting ranking to allocate our inspectors to inspections for $n$ inspections (we take $n$ to be the number of inspections the inspectors were able to do in, say, a month). We then compare the number of failures correctly classified in the top $n$ inspections in the ranking (since there are what would've been done using our best model and methodology) to the number of failures actually found in this same timeframe. If it is greater, then we should be confident in how the model is allocating the City of Chicago's resources. (N.B. of course, the same number of failures will all be discovered over time, but we care about early intervention from a public health perspective!).

Also note that a 2-class approach is sufficient here - we care about failing (vs. not failing). 'Pass' vs. 'Pass w/ Conditions' is an unimportant distinction for how this model is motivated.

In [2]:
import datetime
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest 
from sklearn.ensemble import AdaBoostClassifier as AdaBoost
from sklearn.svm import SVC

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
dataset = pd.read_csv('./data/data_ready.csv')

In [4]:
# convert to access the year conveniently
dataset['inspection_date'] = pd.to_datetime(dataset['inspection_date'])
dataset['inspection_date'] = [day.date() for day in dataset['inspection_date']]

In [5]:
print dataset['inspection_date'].min()
print dataset['inspection_date'].max()

2010-01-04
2016-12-02


In [6]:
# DONT RUN THIS, adjusted 0, 1 labeling up above
# swap 0s and 1s to make neg log loss scoring functions more easy to interpret (we care about fails)
# dataset.replace(to_replace = {'result_binary': {0: 'not pass', 1: 'pass'}}, inplace = True)
# dataset.replace(to_replace = {'result_binary': {'not pass': 1, 'pass': 0}}, inplace = True)

In [7]:
# confirm this 
# dataset.loc[0:10, ['result_binary', 'results']]

In [8]:
np.unique(np.array(map(lambda d: d.year, dataset['inspection_date'].values)))

array([2010, 2011, 2012, 2013, 2014, 2015, 2016])

In [10]:
# withold 2010 for lookback period to build previous features
# train on 2011-2015
train = dataset[map(lambda d: (d.year != 2010) & (d.year != 2016), dataset['inspection_date'].values)] 
# test on 2016 - not this notebook, this is Luke
test = dataset[map(lambda d: d.year == 2016, dataset['inspection_date'].values)] 

x_train = train.drop(['result_binary', 'results', 'inspection_date'], axis = 1) # do we want inspection_date anymore? annoying with sklearn functions...
y_train = train['result_binary']
x_test = test.drop(['result_binary', 'results', 'inspection_date'], axis = 1) 
y_test = test['result_binary']

In [11]:
print dataset.shape, train.shape, test.shape
print 'n.b.: not including observations from 2010'

(120304, 93) (87180, 93) (15742, 93)
n.b.: not including observations from 2010


In [12]:
# correct class imbalance in the train data
print 'Count pass:', sum(y_train == 0)
print 'Count not pass:', sum(y_train == 1)

Count pass: 68510
Count not pass: 18670


In [13]:
from sklearn import preprocessing

In [14]:
preprocessing.scale(x_train['crime'], copy = False)
preprocessing.scale(x_train['sanit'], copy = False)
preprocessing.scale(x_test['crime'], copy = False)
preprocessing.scale(x_test['sanit'], copy = False)

array([ 0.18646092,  2.19330602,  0.19956804, ...,  0.22694735,
        0.22694735, -0.28743414])

In [15]:
## subsampling algorithm 
# takes x_train, y_train and returns a class-balanced x_train, y_train 
# assumes count of class 0 > count of class 1
# subsampled down so a bunch of data disappears - we can change this if we want 
def subsample(x_train, y_train):
    train = pd.concat([x_train, y_train], axis = 1)

    train_0 = train[y_train == 0]
    train_1 = train[y_train == 1]
    
    train_0_subsample = train_0.sample(train_1.shape[0])
    
    train_subsample = pd.concat([train_0_subsample, train_1], axis = 0)
    
    x_train_subsample = train_subsample.iloc[:, :-1]
    y_train_subsample = train_subsample.iloc[:, -1]
    
    return x_train_subsample, y_train_subsample
    
x_train_sub, y_train_sub = subsample(x_train, y_train)

In [16]:
print 'Count pass:', sum(y_train_sub == 0)
print 'Count not pass:', sum(y_train_sub == 1)

Count pass: 18670
Count not pass: 18670


In [17]:
x_train = x_train_sub
y_train = y_train_sub

#### 1. Train for visualization without selection by log loss -- produced for poster -- skip below for selection

In [ ]:
p_hats = {}

In [ ]:
import scipy as sp

# true is list of true classes, pred is list of predicted class probabilities 
def score(y, p_hat):
    p_hat = map(lambda t: t[1], p_hat) # p_hat returned from sklearn is a list of lists with p for both classes, we want p for class 1 or fail 
    epsilon = 1e-15
    p_hat = sp.maximum(epsilon, p_hat)
    p_hat = sp.minimum(1-epsilon, p_hat)
    logloss = sum(y*sp.log(p_hat) + sp.subtract(1,y)*sp.log(sp.subtract(1,p_hat)))
    logloss = logloss * -1.0/len(y)
    return logloss

In [18]:
def naive_model(clf_instance):
    clf_instance.fit(x_train, y_train)
    print 'accuracy:', round(clf_instance.score(x_test, y_test), 2)
    p_hat = clf_instance.predict_proba(x_test)

    print '\n', 'Confusion matrix:'
    conf = confusion_matrix(y_test, clf_instance.predict(x_test))
    conf = conf / float(conf.sum())
    print conf

    print 'log loss:', round(score(y_test, p_hat), 2)
    return map(lambda t: t[1], p_hat) # p_hat returned from sklearn is a list of lists with p for both classes, we want p for class 1 or fail 

In [ ]:
# logreg
logreg = LogReg(C = 1.0, class_weight = 'balanced') 
naive_model(logreg)

In [ ]:
alphas = map(lambda x: 10**x, np.linspace(-2, 2, 5))
logreg_hyperparams = {'C': alphas}  
logreg_labels = ['LogReg, C = 0.10','LogReg, C = 0.1','LogReg, C = 1','LogReg, C = 10','LogReg, C = 100']

In [ ]:
for i, alpha in enumerate(alphas):
    logreg = LogReg(C = alpha, class_weight = 'balanced') 
    p_hats[logreg_labels[i]] = naive_model(logreg)

In [ ]:
# lda
lda = LDA() 
naive_model(lda)

In [ ]:
shrinkage_params = np.linspace(0.0, 1.0, 5)
# lda_hyperparams = {'shrinkage': shrinkage}  
lda_labels = ['LDA, shrinkage = 0.0','LDA, shrinkage = 0.25','LDA, shrinkage = 0.50','LDA, shrinkage = 0.75','LDA, shrinkage = 1.0']

In [ ]:
for i, shrinkage_param in enumerate(shrinkage_params):
    lda = LDA(shrinkage = shrinkage_param, solver = 'lsqr') 
    p_hats[lda_labels[i]] = naive_model(lda)

In [ ]:
# qda
qda = QDA() 
naive_model(qda)

In [ ]:
reg_params = np.linspace(0.0, 1.0, 5)
qda_labels = ['QDA, reg = 0.0','QDA, reg = 0.25','QDA, reg = 0.50','QDA, reg = 0.75','QDA, reg = 1.0']

In [ ]:
for i, reg_param in enumerate(reg_params):
    qda = QDA(reg_param = reg_param) 
    p_hats[qda_labels[i]] = naive_model(qda)

In [ ]:
# knn
knn = KNN(n_neighbors = 5)
naive_model(knn)

In [ ]:
ns = [5,10,25,50]
knn_labels = ['KNN, n = 5','KNN, n = 10','KNN, n = 25','KNN, n = 50']

In [ ]:
for i, n in enumerate(ns):
    knn = KNN(n_neighbors = n) 
    p_hats[knn_labels[i]] = naive_model(knn)

In [ ]:
# bagging - rf
# non-cross validated
rf = RandomForest()
naive_model(rf)

In [ ]:
x_train.shape[1] / 2

In [ ]:
# cross validated
rf_hyperparams = {'n_estimators': [25,50],
                  'max_features': [x_train.shape[1] / 2, x_train.shape[1]], 
                  'max_depth': [2, 4]}  


In [ ]:
rfs = [(25,x_train.shape[1] / 2),
       (25,x_train.shape[1]), 
       (50,x_train.shape[1] / 2), 
       (50,x_train.shape[1])]

rf_labels = ['RF, n = 25, features = 45','RF, n = 25, features = 90','RF, n = 50, features = 45','RF, n = 50, features = 90']

In [ ]:
for i, rf in enumerate(rfs):
    rf = RandomForest(n_estimators = rf[0], max_features = rf[1]) 
    p_hats[rf_labels[i]] = naive_model(rf)

In [ ]:
# boosting - adaboost
# non-cross validated
ada = AdaBoost()
naive_model(ada)

In [ ]:
adas = [25,50,100]

ada_labels = ['AdaBoost, n = 25','AdaBoost, n = 50','AdaBoost, n = 100',]

In [ ]:
for i, ada in enumerate(adas):
    adaboost = RandomForest(n_estimators = ada) 
    p_hats[ada_labels[i]] = naive_model(adaboost)

In [ ]:
p_hats['inspection_date'] = test['inspection_date']
p_hats['result_binary'] = test['result_binary']

In [ ]:
# save this down
output = pd.DataFrame(p_hats)

In [ ]:
output.to_csv('./data/pred_prob_for_fails.csv')

In [20]:
# svm 
# non-cross validated
svc = SVC(probability = True)
pred_prob_svc = naive_model(svc)

In [ ]:
#SLOW
# grid CV
Cs = map(lambda t: 10**t, range(0, 4))
kernels = ['linear', 'poly', 'rbf']
svc_hyperparams = {'C': Cs, 'kernel': kernels}

svc = SVC()
gs_svc = GridSearchCV(svc, param_grid = svc_hyperparams)
gs_svc.fit(x_train, y_train)
p_hat = gs_clf.best_estimator_.predict_proba(x_test)

print score(y_test, p_hat)

#### 2. Tune multiple model classes by cross validating on log loss -- for final output

#### Set hyperparameter space

In [134]:
alphas = map(lambda x: 10**x, np.linspace(-4, 4, 9))
logreg_hyperparams = {'C': alphas}  

shrinkage_params = np.linspace(0.0, 1.0, 5)
lda_hyperparams = {'shrinkage': shrinkage_params, 'solver': ['lsqr']}  

reg_params = np.linspace(0.0, 1.0, 5)
qda_hyperparams = {'reg_param': reg_params}

knn_hyperparams = {'n_neighbors': [5,10,25,50]}

rf_hyperparams = {'n_estimators': [25,50],
                  'max_features': [x_train.shape[1] / 2, x_train.shape[1]], 
                  'max_depth': [2, 4]}  

ada_hyperparams = {'n_estimators': [25,50,100]} 

model_classes = [LogReg, LDA, QDA, KNN, RandomForest, AdaBoost]
model_classes_str = ['LogReg', 'LDA', 'QDA', 'KNN', 'RandomForest', 'AdaBoost']
model_class_hyperparams = [logreg_hyperparams, lda_hyperparams, qda_hyperparams, knn_hyperparams, rf_hyperparams, ada_hyperparams]


#### Return best predictions on the test set

In [171]:
p_hats_cv = {} # contains predicted probabilities for each of best models per model class 
clfs = [] # contains best model per model class

for i, model_class in enumerate(model_classes):
    gs_clf = GridSearchCV(model_class(), param_grid = model_class_hyperparams[i], scoring = 'neg_log_loss', cv = 5)
    gs_clf.fit(x_train, y_train)
    best_clf = gs_clf.best_estimator_
    
    score = 'log loss: ' + str(round(-1 * gs_clf.best_score_, 2))
    
    # best predictor label - class, score, and the tuned hyperparameters of interest 
    best_pred_label = model_classes_str[i] + ', ' + score + ', params = ' + \
                    repr([key + ': ' + str(gs_clf.best_estimator_.get_params()[key]) for key in model_class_hyperparams[i].keys()])
    
    # best predictor's predicted probabilities for class 1 fail 
    best_pred_prob = map(lambda t: t[1], best_clf.predict_proba(x_test))    
    
    # store best predictor's probabilities 
    p_hats_cv[best_pred_label] = best_pred_prob
    
    # store classifiers as well
    clfs.append(best_clf)

In [172]:
print 'Grid search selected models:'
p_hats_cv.keys()

Grid search selected models:


["KNN, log loss: 17.24, params = ['n_neighbors: 50']",
 "RandomForest, log loss: 0.94, params = ['n_estimators: 50', 'max_features: 45', 'max_depth: 2']",
 "QDA, log loss: 1.07, params = ['reg_param: 0.25']",
 "AdaBoost, log loss: 0.73, params = ['n_estimators: 100']",
 "LogReg, log loss: 0.75, params = ['C: 1.0']",
 "LDA, log loss: 0.72, params = ['shrinkage: 0.0', 'solver: lsqr']"]

In [139]:
p_hats_cv['inspection_date'] = test['inspection_date']
p_hats_cv['result_binary'] = test['result_binary']
output = pd.DataFrame(p_hats_cv)
output.to_csv('./data/pred_prob_for_fails_cv.csv')

In [141]:
# test with this 
# gs_clf = GridSearchCV(LogReg(), param_grid = logreg_hyperparams, scoring = 'neg_log_loss', cv = 5)
# gs_clf.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.0001, 0.001, 0.01, 0.10000000000000001, 1.0, 10.0, 100.0, 1000.0, 10000.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring='neg_log_loss',
       verbose=0)

In [150]:
# use to get best n classifiers rather than best classifier 
#
# takes a fit grid searched classifier and returns best n classifiers' hyperparameters
# def best_params(gs_clf, n): 
#     best_ids = np.array(map(lambda t: t[1], np.array(gs_clf.grid_scores_))).argsort()[::-1][:n]
#     best_params = [gs_clf.grid_scores_[best_id][0] for best_id in best_ids]
#     return best_params